<a href="https://colab.research.google.com/github/beautiful-doggie/create_ppt/blob/main/load_deepseek_coder_in_8bits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet bitsandbytes
!pip install --quiet --upgrade transformers # Install latest version of transformers
!pip install --quiet --upgrade accelerate
!pip install --quiet sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3

In [ ]:
!pip install transformers bitsandbytes accelerate

In [ ]:
model_name='deepseek-ai/deepseek-coder-6.7b-base'

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


In [ ]:
model_8bit=AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    load_in_8bit=True,
    torch_dtype=torch.float16
)
tokenizer=AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/793 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

In [ ]:
import json
import re
def generate_teaching_slide_to_json(topic="红黑树",model_name="deepseek-ai/deepseek-coder-6.7b-base",output_dir="./slides")

In [ ]:



prompt = ('''
    你是一位计算机专业课程讲师，正在为《数据结构》课程准备一页关于“红黑树”的幻灯片。

请以结构化的方式输出一页完整的幻灯片内容，包括以下部分：
1. 幻灯片标题
2. 内容提纲（用 bullet points 简明扼要列出）
3. 每个知识点的简要讲解（适合课堂讲解）
4. 对应的代码示例（简洁、注释清晰）
5. 幻灯片中建议插入的示意图说明（如有）

请使用 JSON 格式输出，字段包括：
- "title"：字符串，幻灯片标题
- "outline"：数组，列出主要讲点
- "explanation"：字符串，简洁明了的概括性讲解
- "code_example"：字符串，简洁红黑树代码（插入注释）
- "image_suggestion"：字符串，建议插图内容（如红黑树结构示意图）

请使用中文。'''
)


inputs = tokenizer(prompt, return_tensors="pt", padding=True)
inputs = {k: v.to(model_8bit.device) for k, v in inputs.items()}

outputs = model_8bit.generate(
    **inputs,
    pad_token_id=tokenizer.eos_token_id
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))



    你是一位计算机专业课程讲师，正在为《数据结构》课程准备一页关于“红黑树”的幻灯片。

请以结构化的方式输出一页完整的幻灯片内容，包括以下部分：
1. 幻灯片标题
2. 内容提纲（用 bullet points 简明扼要列出）
3. 每个知识点的简要讲解（适合课堂讲解）
4. 对应的代码示例（简洁、注释清晰）
5. 幻灯片中建议插入的示意图说明（如有）

请使用 JSON 格式输出，字段包括：
- "title"：字符串，幻灯片标题
- "outline"：数组，列出主要讲点
- "explanation"：字符串，简洁明了的概括性讲解
- "code_example"：字符串，简洁红黑树代码（插入注释）
- "image_suggestion"：字符串，建议插图内容（如红黑树结构示意图）

请使用中文。

## 示例

```
{
    "title": "红黑树",
    "outline": [
        "红黑树的定义",
        "红黑树的性质",
        "红黑树的插入",
        "红黑树的删除"
    ],
    "explanation": "红黑树是一种自平衡的二叉搜索树，它在插入、删除和查找操作中保持平衡，从而提高了查找效率。",
    "code_example": "// 红黑树的定义\nstruct RBTree {\n    int key;\n    int value;\n    int color;\n    RBTree *left;\n    RBTree *right;\n    RBTree *parent;\n};\n\n// 红黑树的插入\nvoid insert(RBTree *root, int key, int value) {\n    RBTree *node = new RBTree(key, value);\n    RBTree *parent = nullptr;\n    RBTree *cur = root;\n    while (cur != nullptr) {\n        parent = cur;\n        if (key < cur->key) {\n            cur = cur->left;\n       